O projeto consiste em uma conciliação do valor total de vendas de recarga de celular por operadora e por loja. A Bemol recebe um arquivo de uma empresa financeira (detalhamento_m4u.xlsx) onde possui o valor consolidado de vendas e o objetivo é verificar se os dados do SAP (mcg3.csv) estão de acordo com o informado. Gerar um alerta caso os valores estejam divergentes.

**Dica:** Os dados do SAP vêm separados por cada transição de venda realizada. A ideia é de agrupar esses dados para deixá-los no formato parecido com que recebemos da empresa externa e, assim, poder unificar as tabelas e fazer a comparação.

In [0]:
import pandas as pd

#1. Leitura dos arquivos .csv e .xlsx

In [0]:
df_dadossap = pd.read_csv('mcg3.csv', sep = '|')

In [0]:
df_detalhamento = pd.concat(pd.read_excel('detalhamento_m4u.xlsx', sheet_name = None,skiprows = 1,ignore_index=False))

#2. Tratamento dos Dados

In [5]:
df_dadossap.columns = ['escritorio_vendas','fornecedor','material','data','valor_liq']
df_dadossap

,escritorio_vendas,fornecedor,material,data,valor_liq
0,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",02.01.2020,20.0
1,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",30.12.2019,10.0
2,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",04.01.2020,20.0
3,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",02.01.2020,10.0
4,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",03.01.2020,25.0
...,...,...,...,...,...
5108,701,CLARO,"131317 RECARGA VIRTUAL CLARO R$ 1,00",30.12.2019,20.0
5109,701,CLARO,"131317 RECARGA VIRTUAL CLARO R$ 1,00",04.01.2020,20.0
5110,701,CLARO,"131317 RECARGA VIRTUAL CLARO R$ 1,00",05.01.2020,20.0
5111,701,CLARO,"131317 RECARGA VIRTUAL CLARO R$ 1,00",05.01.2020,45.0


In [7]:
df_detalhamento.columns = ['nome_loja','escritorio_vendas','UF','operadora','v_bruto']
df_detalhamento

nome_loja  escritorio_vendas  UF operadora  v_bruto
AM 0          BEMOL AVENIDA                103  AM     Claro     2651
   1          BEMOL AVENIDA                103  AM        Oi     3309
   2          BEMOL AVENIDA                103  AM       Tim     1550
   3          BEMOL AVENIDA                103  AM      Vivo     4739
   4          BEMOL BARROSO                107  AM     Claro     1495
...                     ...                ...  ..       ...      ...
RO 23  LOJA BEMOL JI-PARANA                205  RO     Claro      123
AC 0       BEMOL RIO BRANCO                401  AC     Claro      500
   1       BEMOL RIO BRANCO                401  AC        Oi      375
   2       BEMOL RIO BRANCO                401  AC       Tim      125
   3       BEMOL RIO BRANCO                401  AC      Vivo      343

[162 rows x 5 columns]

In [0]:
df_dadossap['valor_liq'] = df_dadossap['valor_liq'].astype(int)

In [0]:
df_dadossap = df_dadossap.drop(['material','data'],axis='columns')

In [0]:
df_dadossap

In [11]:
df_detalhamento = df_detalhamento.drop(['UF'],axis='columns')
df_detalhamento

nome_loja  escritorio_vendas operadora  v_bruto
AM 0          BEMOL AVENIDA                103     Claro     2651
   1          BEMOL AVENIDA                103        Oi     3309
   2          BEMOL AVENIDA                103       Tim     1550
   3          BEMOL AVENIDA                103      Vivo     4739
   4          BEMOL BARROSO                107     Claro     1495
...                     ...                ...       ...      ...
RO 23  LOJA BEMOL JI-PARANA                205     Claro      123
AC 0       BEMOL RIO BRANCO                401     Claro      500
   1       BEMOL RIO BRANCO                401        Oi      375
   2       BEMOL RIO BRANCO                401       Tim      125
   3       BEMOL RIO BRANCO                401      Vivo      343

[162 rows x 4 columns]

#3. Correlação das Tabelas

In [14]:
df_dadossap.shape

(5113, 3)

In [13]:
df_detalhamento.shape

(162, 4)

In [0]:
#Agrupamento das tabelas pelo escritorio_vendas e faz a soma das duas colunas
df_dadossap = df_dadossap.groupby(['escritorio_vendas']).agg({'valor_liq':'sum'}).reset_index()

In [16]:
df_dadossap.shape

(42, 2)

In [0]:
#Agrupamento das duas tabelas numa só
df_relatorio = pd.merge(df_dadossap, df_detalhamento, on = ['escritorio_vendas'], how = 'outer')

In [0]:
#Exclui a coluna
df_relatorio = df_relatorio.drop(['escritorio_vendas'],axis='columns')

In [20]:
df_relatorio

,valor_liq,nome_loja,operadora,v_bruto
0,7374,BEMOL MATRIZ,Claro,1527
1,7374,BEMOL MATRIZ,Oi,1730
2,7374,BEMOL MATRIZ,Tim,975
3,7374,BEMOL MATRIZ,Vivo,3142
4,12250,BEMOL AVENIDA,Claro,2651
...,...,...,...,...
157,1021,BEMOL FARMA STUDIO 5,Vivo,380
158,599,BEMOL BOA VISTA,Claro,140
159,599,BEMOL BOA VISTA,Oi,20
160,599,BEMOL BOA VISTA,Tim,135


In [0]:
df_relatorio = df_relatorio.fillna(0)

In [0]:
#Reorganiza a tabela
df_relatorio = df_relatorio[['nome_loja','operadora','valor_liq','v_bruto']]

In [24]:
df_relatorio

,nome_loja,operadora,valor_liq,v_bruto
0,BEMOL MATRIZ,Claro,7374,1527
1,BEMOL MATRIZ,Oi,7374,1730
2,BEMOL MATRIZ,Tim,7374,975
3,BEMOL MATRIZ,Vivo,7374,3142
4,BEMOL AVENIDA,Claro,12250,2651
...,...,...,...,...
157,BEMOL FARMA STUDIO 5,Vivo,1021,380
158,BEMOL BOA VISTA,Claro,599,140
159,BEMOL BOA VISTA,Oi,599,20
160,BEMOL BOA VISTA,Tim,599,135


#4. Regra de Negócio

In [0]:
#se valor liquido != valor bruto  gerar mensagem de alerta

def verificar_status(valor_liq,v_bruto):
  if (valor_liq != v_bruto):
    return 'Alerta'
  else:
    return ''

In [0]:
#Criação da nova Coluna 'status'
df_relatorio['status'] = df_relatorio.apply(lambda row: verificar_status(row['valor_liq'],row['v_bruto']), axis = 'columns')

In [32]:
df_relatorio

,nome_loja,operadora,valor_liq,v_bruto,status
0,BEMOL MATRIZ,Claro,7374,1527,Alerta
1,BEMOL MATRIZ,Oi,7374,1730,Alerta
2,BEMOL MATRIZ,Tim,7374,975,Alerta
3,BEMOL MATRIZ,Vivo,7374,3142,Alerta
4,BEMOL AVENIDA,Claro,12250,2651,Alerta
...,...,...,...,...,...
157,BEMOL FARMA STUDIO 5,Vivo,1021,380,Alerta
158,BEMOL BOA VISTA,Claro,599,140,Alerta
159,BEMOL BOA VISTA,Oi,599,20,Alerta
160,BEMOL BOA VISTA,Tim,599,135,Alerta


#5. Exportando Dados no arquivo

In [0]:
# .csv
df_relatorio.to_csv('Relatório Final Do Desafio.csv', sep='|', index=False)

In [0]:
# .xlsx
writer = pd.ExcelWriter('Relatório Final Do Desafio.xlsx')

df_relatorio.to_excel(writer, sheet_name='relatorio', index=False)

writer.save()